build_dataset 해당

In [1]:
# from agents.for_model import MacroAData
# 
# macro_agent = MacroAData()
# macro_agent.fetch_data()
# macro_agent.add_features()
# macro_agent.save_csv()
# macro_agent.make_close_price()

$$$$   X = macro_sercher(agent, ticker) 역할

In [2]:
from agents.macro_agent import MacroPredictor
from datetime import datetime

macro_sub = MacroPredictor(
    agent_id='MacroSentiAgent',
    base_date=datetime.today(),
    window=40,
    ticker="NVDA"
)
macro_sub.load_assets()               # 모델, 스케일러 등 불러오기


[INFO] 모델 및 스케일러 로드 중...
[OK] 모델 및 스케일러 로드 완료


In [3]:
macro_sub.fetch_macro_data()          # macro_df 불러오기
X_seq, X_scaled = macro_sub.prepare_features()  # 입력 시퀀스 준비

[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed

[MacroSentimentAgent] Stock data: (41, 9)
[MacroSentimentAgent] Data shape: (41, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (41, 190)
[OK] 매크로 데이터 수집 완료: (41, 191)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 189 / 스케일러 기준 피처 수: 189
[OK] 스케일링 및 시퀀스 변환 완료


In [4]:
print(f"X_seq: {X_seq}")
print(f"X_scaled: {X_scaled}")

X_seq: [[[-0.35126372 -0.40248558  1.08025784 ... -0.66549374  3.49979922
    3.57621585]
  [-0.26829567 -0.27227473  0.94319106 ... -0.117693    3.49979922
    3.57621585]
  [-0.35772235 -0.31567835  0.98509405 ...  0.08960864  3.49979922
    3.57621585]
  ...
  [-0.55048626 -0.18199578  0.89591085 ...  0.21733271  3.79123894
    3.83391188]
  [-0.45857555 -0.17852275  0.91234111 ...  0.57341406  3.80727521
    3.84214434]
  [-0.4729832  -0.212204    0.93575827 ...  0.7376518   3.8539599
    3.85056271]]]
X_scaled:     Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X  Open_GC=F  Open_HG=F  Open_QQQ  \
0   -0.335862      -0.364290       1.021317   5.619121   1.019029  2.955674   
1   -0.351264      -0.402486       1.080258   5.819403   1.104640  2.784059   
2   -0.268296      -0.272275       0.943191   6.079545   1.187079  2.885871   
3   -0.357722      -0.315678       0.985094   6.061097   1.068969  2.905282   
4   -0.382066      -0.300054       0.979030   6.128486   1.106225  3.030619   
5   

$$$$   macro_predictor(X) 역할

In [5]:
pred_prices, target = macro_sub.m_predictor(X_seq)

[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
NVDA: 마지막 종가=182.89 → 예측 종가=179.19 (예상 수익률 -2.02%)
[WARN] scaler_y inverse_transform 실패: non-broadcastable output operand with shape (3,1) doesn't match the broadcast shape (3,3)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA     182.888          179.191           -0.0202      -2.0215   

   uncertainty  confidence  
0       0.2068      4.8364  

================= 예측 결과 (값) =================
{'NVDA': 179.19100079999282}


In [6]:
print(f"pred_prices: {pred_prices}")
print(f"target: {target}")

pred_prices: {'NVDA': 179.19100079999282}
target: Target(next_close=179.191, uncertainty=0.14566712081432343, confidence=4.8364, feature_cols=None, importances=None)


llm 생성 용.. macro_reviewer_draft

In [7]:
 total_json, opinion = macro_sub.macro_reviewer_draft(X_scaled, pred_prices, target)
 

1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading AAPL ...


[*********************100%***********************]  1 of 1 completed


   ↳ Downloading MSFT ...


...

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\shap\explainers\_kernel.py:708: UserWarning: Linear regression equation is singular, a least squares solutions is used instead.
To avoid this situation and get a regular matrix do one of the following:
1) turn up the number of samples,
2) turn up the L1 regularization with num_features(N) where N is less than the number of samples,
3) group features together to reduce the number of inputs that need to be explained.
  warnings.warn(


[Causal SHAP] Computing causal perturbation effects...
[Interaction SHAP] Computing interaction correlations...


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 8 iterations, i.e. alpha=3.937e-02, with an active set of 7 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\shap\explainers\_kernel.py:708: UserWarning: Linear regression equation is singular, a least squares solutions is used instead.
To avoid this situation and get a regular matrix do one of the following:
1) turn up the number of samples,
2) turn up the L1 regularization with num_features(N) where N is less than the number of samples,
3) group features together to reduce the number of inputs that need to be explained.
  warnings.warn(


[Interaction SHAP] Computing correlation among 6 features...

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 179.19100079999282} =================

================= LLM Explanation =================
(1) Temporal 분석에서 NVDA 관련 변수들의 영향력은 매우 미미하며, 시간에 따른 피처 영향 변화가 거의 없음을 확인했다. 이는 NVDA 주가 예측에 외부 변수들의 직접적 영향이 제한적임을 시사한다. (2) Causal 분석에서는 금 관련 저가(Low_GC=F)와 QQQ 저가(Low_QQQ)가 주가 상승에 긍정적 인과 효과를 보였고, AAPL 5일 이동평균과 SPY 종가도 상승 요인으로 작용했다. 반면, QQQ 종가와 다우 지수 저가(Low_^DJI), 그리고 S&P 500 지수 고가(High_^GSPC)는 하락 요인으로 나타났다. 이는 금 가격과 기술주 지수의 변동성이 NVDA 및 관련 기술주에 영향을 미치며, 금리 및 시장 변동성 지표가 복합적으로 작용함을 의미한다. (3) Interaction 분석에서는 Open_QQQ와 Open_^GSPC, High_GC=F, High_HG=F 간에 강한 상호작용이 존재하며, 이는 기술주 지수와 금속 가격 간의 상관관계가 NVDA 주가에 영향을 줄 수 있음을 시사한다. (4) 종합 결론으로 NVDA는 금 가격과 기술주 지수의 변동에 민감하며, 금 가격 상승과 QQQ 지수 저가가 주가 상승을 견인하는 주요 변수로 작용한다. 반면 AAPL과 MSFT는 상대적으로 시장 지수 및 금리 변동에 더 민감한 경향이 있어 NVDA와 차별화된 변수 민감도를 보인다. 따라서 NVDA의 예측 종가 179.191은 금 가격 상승과 기술주 지수의 긍정적 신호에 기반한 상승 전망으로 해석된다.


In [8]:
print(f"total_json: {total_json}")
print(f"opinion: {opinion}")

total_json: {'agent_id': 'MacroSentiAgent', 'target': Target(next_close=179.191, uncertainty=0.14566712081432343, confidence=4.8364, feature_cols=None, importances=None), 'reason': '(1) Temporal 분석에서 NVDA 관련 변수들의 영향력은 매우 미미하며, 시간에 따른 피처 영향 변화가 거의 없음을 확인했다. 이는 NVDA 주가 예측에 외부 변수들의 직접적 영향이 제한적임을 시사한다. (2) Causal 분석에서는 금 관련 저가(Low_GC=F)와 QQQ 저가(Low_QQQ)가 주가 상승에 긍정적 인과 효과를 보였고, AAPL 5일 이동평균과 SPY 종가도 상승 요인으로 작용했다. 반면, QQQ 종가와 다우 지수 저가(Low_^DJI), 그리고 S&P 500 지수 고가(High_^GSPC)는 하락 요인으로 나타났다. 이는 금 가격과 기술주 지수의 변동성이 NVDA 및 관련 기술주에 영향을 미치며, 금리 및 시장 변동성 지표가 복합적으로 작용함을 의미한다. (3) Interaction 분석에서는 Open_QQQ와 Open_^GSPC, High_GC=F, High_HG=F 간에 강한 상호작용이 존재하며, 이는 기술주 지수와 금속 가격 간의 상관관계가 NVDA 주가에 영향을 줄 수 있음을 시사한다. (4) 종합 결론으로 NVDA는 금 가격과 기술주 지수의 변동에 민감하며, 금 가격 상승과 QQQ 지수 저가가 주가 상승을 견인하는 주요 변수로 작용한다. 반면 AAPL과 MSFT는 상대적으로 시장 지수 및 금리 변동에 더 민감한 경향이 있어 NVDA와 차별화된 변수 민감도를 보인다. 따라서 NVDA의 예측 종가 179.191은 금 가격 상승과 기술주 지수의 긍정적 신호에 기반한 상승 전망으로 해석된다.'}
opinion: Opinion(agent_id='MacroSentiAgent', target=Target(next_cl

reviewer_rebut

In [9]:
# reviewer_rebut()

토론 테스트

In [10]:
from debate_ver4.agents_tmp.debate_agent import DebateAgent

debate_agent = DebateAgent(1, "NVDA")

In [11]:
debate_agent.run_dataset()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (466, 14, 10), y_seq: (466, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (466 samples, 10 features)
[MacroSentiAgent] X_seq: (466, 14, 13), y_seq: (466, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (466 samples, 13 features)
[SentimentalAgent] X_seq: (466, 14, 8), y_seq: (466, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (466 samples, 8 features)


In [12]:
debate_agent.get_opinion(0, "NVDA")

MacroSentiAgent의 데이터 로드.. macro_sercher
[INFO] 모델 및 스케일러 로드 중...
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Stock data: (41, 9)
[MacroSentimentAgent] Data shape: (41, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (41, 190)
[OK] 매크로 데이터 수집 완료: (41, 191)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 189 / 스케일러 기준 피처 수: 189
[OK] 스케일링 및 시퀀스 변환 완료
■ macro_sercher StockData 생성 완료 (NVDA)
MacroSentiAgent의 예측
[INFO] 예측 수행 중...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
NVDA: 마지막 종가=182.89 → 예측 종가=179.19 (예상 수익률 -2.02%)
[WARN] scaler_y inverse_transform 실패: non-broadcastable output operand with shape (3,1) doesn't match the broadcast shape (3,3)

================= 예측 결과 (표) =================
  Ticker  Last_Close  Predicted_Close  Predicted_Return  Predicted_%  \
0   NVDA     182.888         179.1911           -0.0202      -2.0214   

   uncertainty  confidence  
0       0.1517        6.59  

================= 예측 결과 (값) =================
{'NVDA': 179.1910747222248}
1️⃣ Collecting macro features (15 tickers)...


...

[Causal SHAP] Computing causal perturbation effects...
[Interaction SHAP] Computing interaction correlations...


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 1 iterations, i.e. alpha=9.906e-04, with an active set of 1 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=9.001e-04, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
C:\Users\SPOTV\IdeaProject_venv\capstone_project\Lib\site-packages\sklearn\linear_model\_least_angle.py:723: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=8.547e-04, wi

[Interaction SHAP] Computing correlation among 4 features...

4️⃣ Generating explanation using LLM...

================= pred_prices:{'NVDA': 179.1910747222248} =================

================= LLM Explanation =================
(1) Temporal 분석: 최근 시점에서 변수들의 영향력이 전반적으로 매우 미미하며, 대부분의 피처 영향력이 0에 가까워 변화가 거의 없습니다. 다만 Close_^IXIC 변수의 영향력이 약간 증가하는 경향이 보이나 전체적으로 큰 변화는 없습니다. 이는 최근 시장 변동성이 낮거나 모델이 단기 변동성보다는 장기적 추세에 더 의존함을 시사합니다.

(2) Causal 분석: NVDA 주가에 인과적으로 긍정적 영향을 미치는 주요 변수는 Low_QQQ, Adj Close_SPY, Adj Close_GC=F, Low_GC=F, AAPL_ma5 등으로 나타났습니다. 이는 기술주 및 주요 지수의 저점 지지가 NVDA 주가 상승에 긍정적 역할을 함을 의미합니다. 반면 Low_^DJI와 Adj Close_QQQ는 음의 인과 효과를 보여 일부 시장 지표의 하락이 NVDA에 부정적 영향을 줄 수 있음을 시사합니다. 예를 들어, "기술주 지수(QQQ) 저점 상승(+) → 투자 심리 개선 → NVDA 주가 상승" 구조로 해석할 수 있습니다.

(3) Interaction 분석: Open_USDJPY=X와 Adj Close_QQQ, Close_HG=F_ret, Close_QQQ_ret 간 상호작용이 관찰됩니다. 이는 환율 변동과 기술주 지수, 금 선물 수익률 간의 상호작용이 NVDA 주가에 복합적 영향을 미칠 수 있음을 의미합니다. 예를 들어, "달러 강세(Open_USDJPY=X 상승)와 기술주 지수 변동(Adj Close_QQQ) 상호작용 → 수출입 기업 실적 및 투자 

{'MacroSentiAgent': Opinion(agent_id='MacroSentiAgent', target=Target(next_close=179.1911, uncertainty=0.15444950759410858, confidence=6.59, feature_cols=None, importances=None), reason='모델은 다음 거래일 종가가 현재가 대비 약 0.15% 상승한 179.191로 예측하고 있습니다. 기술적 분석 관점에서 QQQ와 SPY의 저점이 강하게 지지되고 있으며, 금(GC) 관련 지표도 긍정적인 영향을 미치고 있습니다. 특히, QQQ의 저점과 SPY의 조정 종가가 상승 모멘텀을 촉발하는 신호로 작용하고 있습니다. 이러한 인과 관계와 상호작용을 고려할 때, 단기적으로 주가 상승이 기대됩니다.')}